In [ ]:
import pandas as pd
import os
import zipfile
import json
import io
from IPython.display import display

In [ ]:
zip_file_path = 'datatest (1).zip'
folder_path = "root/workspace/data-eng-test"
data = {}
output_file = "output_data.json"


In [ ]:
def generate_schema(zip_path, output_file=None):
    schema = {}

    with zipfile.ZipFile(zip_path, "r") as zip_file:
        for zip_info in zip_file.infolist():
            if zip_info.filename.endswith(".json"):
                with zip_file.open(zip_info) as json_file:
                    data = json.load(io.TextIOWrapper(json_file, encoding="utf-8"))

                for key in data.keys():
                    if key not in schema:
                        schema[key] = {}

                    if isinstance(data[key], dict):
                        for subkey in data[key].keys():
                            if subkey not in schema[key]:
                                schema[key][subkey] = type(data[key][subkey]).__name__
                    elif isinstance(data[key], list):
                        for element in data[key]:
                            if isinstance(element, dict):
                                for subkey in element.keys():
                                    if subkey not in schema[key]:
                                        schema[key][subkey] = type(element[subkey]).__name__

    schema = {k: {sk: v for sk, v in sorted(sv.items())} for k, sv in schema.items()}
    schema = dict(sorted(schema.items()))

    return schema
    # Write the schema to the output file
    with open(output_file, "w") as output:
        json.dump(schema, output, indent=4)


In [ ]:
base_schema = generate_schema(zip_file_path)

In [ ]:
def retrun_header(key, data):
    return list(data[key].keys())

In [ ]:
assortment = retrun_header("assortment", base_schema)
images = retrun_header("images", base_schema)
marketplace = retrun_header("marketplace", base_schema)
reviews = retrun_header("reviews", base_schema)
variants = retrun_header("variants", base_schema)

In [ ]:
def data_divider_generator(zip_file_path,output_file=None) :
    schema = {}

    with zipfile.ZipFile(zip_file_path, "r") as zip_file:
        for zip_info in zip_file.infolist():
            if zip_info.filename.endswith(".json"):
                with zip_file.open(zip_info) as json_file:
                    data = json.load(io.TextIOWrapper(json_file, encoding="utf-8"))
                for key in data.keys():
                    if key not in schema:
                        schema[key] = []
                    if isinstance(data[key], dict):
                        schema[key].append(data[key])
                    elif isinstance(data[key], list):
                        for element in data[key]:
                            if isinstance(element, dict):
                                schema[key].append(element)

    schema = {k: sorted(v, key=lambda x: list(x.keys())) for k, v in schema.items()}
    schema = dict(sorted(schema.items()))
    return schema
    # with open(output_file, "w") as output:
    #     json.dump(schema, output, indent=4)

In [ ]:
schema = data_divider_generator(zip_file_path)

In [ ]:
assortment_df = pd.DataFrame.from_dict(schema["assortment"])
images_df = pd.DataFrame.from_dict(schema["images"])
marketplace_df = pd.DataFrame.from_dict(schema["marketplace"])
reviews_df = pd.DataFrame.from_dict(schema["reviews"])
variants_df = pd.DataFrame.from_dict(schema["variants"])


In [ ]:
display(assortment_df)
display(images_df)
display(marketplace_df)
display(reviews_df)
display(variants_df)

In [ ]:
merged_AR =pd.merge(assortment_df, marketplace_df, on='idRetailerSKU', how='outer', suffixes=('_assortment_df', '_marketplace_df'))

display(merged_AR)

In [ ]:
filters = ['retailerPrice', 'manufacturerPrice', 'priceVariation', 'available'] 

filtered_AR =  merged_AR.loc[merged_AR['seller_marketplace_df'] == 'Americanas']


display(filtered_AR)

In [ ]:
variation_dataset = filtered_AR[['retailerProductCode','manufacturerTitle','idRetailerSKU','manufacturerPrice', 'retailerPrice','retailerFromPrice','seller_marketplace_df', 'variation', 'available']]
variation_dataset_filtered = variation_dataset.loc[variation_dataset['variation'].notna()]
display(variation_dataset_filtered)

In [ ]:
top_10_price_variations = variation_dataset_filtered.groupby('manufacturerTitle').apply(lambda x: x.sort_values(by='variation', ascending=False).head(1))
top_10_price_variations['price_diff'] = top_10_price_variations['retailerPrice'] - top_10_price_variations['manufacturerPrice']
top_10_price_variations = top_10_price_variations[['manufacturerTitle', 'idRetailerSKU', 'variation', 'price_diff']].sort_values(by='variation', ascending=False).head(10)
display(top_10_price_variations)

In [ ]:
top_10_price_variations = variation_dataset_filtered.groupby('manufacturerTitle').apply(lambda x: x.sort_values(by='variation', ascending=True).head(1))
top_10_price_variations['price_diff'] = top_10_price_variations['retailerPrice'] - top_10_price_variations['manufacturerPrice']
top_10_price_variations = top_10_price_variations[['manufacturerTitle', 'idRetailerSKU', 'variation', 'price_diff']].sort_values(by='variation', ascending=True).head(10)
display(top_10_price_variations)

In [ ]:
availability_df = filtered_AR[['retailerProductCode','manufacturerTitle','idRetailerSKU','manufacturerPrice', 'retailerPrice','retailerFromPrice','seller_marketplace_df', 'variation', 'available']]
availability_df_filtered = availability_df.loc[variation_dataset['available'].notna()]
display(availability_df_filtered)

In [ ]:
top_10_unavailable = availability_df_filtered.sort_values(by='available', ascending=False).groupby('manufacturerTitle').head(1).sort_values(by='available')[:10]

top_10_unavailable['num_false'] = availability_df_filtered.groupby('manufacturerTitle')['available'].transform(lambda x: x.value_counts().get(False, 0))
top_10_unavailable = top_10_unavailable.sort_values(by='num_false', ascending=False)

display(top_10_unavailable)

In [84]:

avg_retailer_price = variation_dataset_filtered.groupby('manufacturerTitle')['retailerPrice'].mean().reset_index()
avg_retailer_price.columns = ['manufacturerTitle', 'avg_retailer_price']


avg_manufacturer_price = variation_dataset_filtered.groupby('manufacturerTitle')['manufacturerPrice'].mean().reset_index()
avg_manufacturer_price.columns = ['manufacturerTitle', 'avg_manufacturer_price']


price_variation = variation_dataset_filtered.groupby('manufacturerTitle').apply(lambda x: (x['retailerPrice'] - x['manufacturerPrice']).mean()).reset_index()
price_variation.columns = ['manufacturerTitle', 'price_variation']


availability_rate = variation_dataset_filtered.groupby('manufacturerTitle')['available'].mean().reset_index()
availability_rate.columns = ['manufacturerTitle', 'availability_rate']


price_variation_rate = variation_dataset_filtered.groupby('manufacturerTitle').apply(lambda x: ((x['retailerPrice'] - x['manufacturerPrice']).abs() / x['manufacturerPrice']).mean()).reset_index()
price_variation_rate.columns = ['manufacturerTitle', 'price_variation_rate']


price_difference = variation_dataset_filtered.groupby('manufacturerTitle').apply(lambda x: (x['retailerPrice'] - x['manufacturerPrice']).mean()).reset_index()
price_difference.columns = ['manufacturerTitle', 'price_difference']


max_price = variation_dataset_filtered.groupby('manufacturerTitle')['retailerPrice'].max().reset_index()
max_price.columns = ['manufacturerTitle', 'max_price']
min_price = variation_dataset_filtered.groupby('manufacturerTitle')['retailerPrice'].min().reset_index()
min_price.columns = ['manufacturerTitle', 'min_price']


num_unavailable = variation_dataset_filtered.groupby('manufacturerTitle')['available'].apply(lambda x: sum(x==False)).reset_index()
num_unavailable.columns = ['manufacturerTitle', 'num_unavailable']


metrics_df = avg_retailer_price.merge(avg_manufacturer_price, on='manufacturerTitle')
metrics_df = metrics_df.merge(price_variation, on='manufacturerTitle')
metrics_df = metrics_df.merge(availability_rate, on='manufacturerTitle')
metrics_df = metrics_df.merge(price_variation_rate, on='manufacturerTitle')
metrics_df = metrics_df.merge(price_difference, on='manufacturerTitle')
metrics_df = metrics_df.merge(max_price, on='manufacturerTitle')
metrics_df = metrics_df.merge(min_price, on='manufacturerTitle')
metrics_df = metrics_df.merge(num_unavailable, on='manufacturerTitle')


display(metrics_df)


,manufacturerTitle,avg_retailer_price,avg_manufacturer_price,price_variation,availability_rate,price_variation_rate,price_difference,max_price,min_price,num_unavailable
0,Calculadora de Mesa MV4122 com Visor 12 Digíto...,31.896552,26.730000,5.166552,1.000000,0.193287,5.166552,32.90,29.99,0
1,Carregador de Pilhas USB Elgin com Cabo Alim 5...,104.817586,142.840000,-38.022414,1.000000,0.266189,-38.022414,119.99,99.99,0
2,Cervejeira Consul Cor Titanium com 82L Display...,2375.704286,2499.000000,-123.295714,0.724138,0.049564,-123.295714,2499.99,2149.99,192
3,Cervejeira Consul Cor Titanium com 82L Display...,2375.207391,2318.000000,57.207391,0.821429,0.046966,57.207391,2499.99,2149.99,145
4,Cooktop 4 bocas Brastemp com Grades Piatina e ...,588.990000,699.000000,-110.010000,0.931034,0.157382,-110.010000,678.99,549.99,54
...,...,...,...,...,...,...,...,...,...,...
71,Pilha Recarregável Elgin AAA-900 mAh Cartela c...,21.990000,55.561429,-33.571429,0.724138,0.497184,-33.571429,21.99,21.99,168
72,Refletor Projetor Led 10W 6500K IP65 Preto Elgin,29.990000,27.200000,2.790000,0.103448,0.102574,2.790000,29.99,29.99,78
73,Refletor Projetor Led 20W Inteligente Elgin Wi...,385.823333,249.900000,135.923333,1.000000,0.543911,135.923333,399.99,314.99,0
74,Refletor Projetor Led 50W 6500K IP65 Preto Elgin,64.472759,59.900000,4.572759,1.000000,0.087750,4.572759,64.99,49.99,0
